# Spotify Recommendation - Finding your music taste!

## Import necessary libraries

In [69]:
import pandas as pd
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import requests
import time
from pandas import json_normalize

## Using Credentials to access OAuth for spotify

In [2]:
# Your credentials
client_id = '0af53b02c90245839d0f6d6f3bf45fb1'
client_secret = '89783cd95eac43eebbfc7f2e2a885dbb'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://accounts.spotify.com/api/token',
    client_id=client_id,
    client_secret=client_secret,
    include_client_id=True
)

## Ideas for model

Before we start, we need to consider a couple of things when recommending songs:

- Is the track available for specified country?

- Did the user dislike similar tracks or this specified track?

Features for the model:

- Maybe consider getting input from users of language/country of song you want?

## Initializing Token and Getting API Request

In [3]:
# The headers to provide the access token for authentication
headers = {
    'Authorization': f'Bearer {token["access_token"]}',     # OAuth 2.0 Bearer token for authorization
    'Content-Type': 'application/json'                      # Indicates the media type of the resource
}

In [4]:
token_expiry = time.time() + token['expires_in'] # gives time of expiry in total seconds
#token_expiry_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(token_expiry)) # converting to readable time format

we need to check if time.time() is going to exceed the token_expiry_date. If so then renew the token!

In [8]:
# void function to get new token if needed
def refresh_token_if_needed():
    global token, token_expiry  # Declare token and token_expiry as global variables

    # Check if the current time is within 60 seconds of expiry or past it
    if time.time() > token_expiry - 60:
        client = BackendApplicationClient(client_id=client_id)
        oauth = OAuth2Session(client=client)
        
        # Fetch a new token and update the global token variable
        token = oauth.fetch_token(
            token_url='https://accounts.spotify.com/api/token',
            client_id=client_id,
            client_secret=client_secret,
            include_client_id=True
        )
        
        # Calculate and update the global token_expiry variable
        token_expiry = time.time() + token['expires_in']


## Recommending songs based on user chosen track

In [17]:
song_title = "Self Care"
artist_name = "Mac Miller"
# check if search is case sensitive?

In [46]:
refresh_token_if_needed()
headers = {
    'Authorization': f'Bearer {token["access_token"]}', # specify token
    'Content-Type': 'application/json' # specify data type that you recieve
}


In [37]:
search_url = "https://api.spotify.com/v1/search" # Sending a GET request to Spotify API, v1 -> using version 1 of the API

Searching song based on song title and artist name

In [50]:
# can put information about these things:
# album
# artist
# track
# year
# upc
# tag:hisper -> return albumns with lowest popularity
# tag:new -> shows new albumns
# genre
search_query1 = f'track:{song_title} artist:{artist_name}'
search_query2 = f'artist:{artist_name}'

In [61]:
# dictionary format for search parameter
# https://developer.spotify.com/documentation/web-api/reference/search 
# use the link above for a documentation guide for search parameters
# 
search_params = {
    'q': search_query1,
    'type': 'track' # can return: "album", "artist", "playlist", "track", "show", "episode", "audiobook"
    }

In [62]:
response = requests.get(search_url, headers=headers, params=search_params) # encoding to acceptable format for Spotify API to understand

In [63]:
search_results = response.json()

In [73]:
df = json_normalize(search_results, record_path=['tracks', 'items'])

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   artists                       20 non-null     object
 1   available_markets             20 non-null     object
 2   disc_number                   20 non-null     int64 
 3   duration_ms                   20 non-null     int64 
 4   explicit                      20 non-null     bool  
 5   href                          20 non-null     object
 6   id                            20 non-null     object
 7   is_local                      20 non-null     bool  
 8   name                          20 non-null     object
 9   popularity                    20 non-null     int64 
 10  preview_url                   20 non-null     object
 11  track_number                  20 non-null     int64 
 12  type                          20 non-null     object
 13  uri                   

In [66]:
tracks_data = search_results['tracks']['items']
df = pd.DataFrame(tracks_data)

In [56]:
if response.status_code == 200:
    search_results = response.json()
    tracks = search_results['tracks']['items']

    if not tracks:
        print("No results found.")
    else:
        top_result = tracks[0]  # Get the top result
        print(f"Top result: {top_result['name']} by {top_result['artists'][0]['name']}")
else:
    print(f"Failed to fetch search results: {response.json()}")

Top result: The Spins by Mac Miller
